In [ ]:
%pip install -U langchain jq

In [ ]:
from langchain.document_loaders import JSONLoader
from pprint import pprint

fields = ["trait", "action", "legendary", "bonus", "descriptions"]
documents_by_field = {}

for field in fields:
    jq_schema = f'keys[] as $k | "\($k), \(.[$k] | .)"'
    loader = JSONLoader(
        file_path='./monster_text.json', jq_schema=jq_schema, content_key=field)
    documents_by_field[field] = loader.load()

pprint(documents_by_field)
